In [19]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/neg_LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the centroids adjusted for the simulation, so the Magbangons combined 
#centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
Centroids <- fread(file="~/oceanography/empirical_data/site_centroids_SimTest.csv") 
Centroids$site <- gsub("_full", "", Centroids$site, fixed=TRUE)
Centroids$site <- gsub("_ten_per_cover", "", Centroids$site, fixed=TRUE)

Centroids <- Centroids %>%
    arrange(site)

#setorder(Centroids, site)#warning! This sets order based on site, and then lat/lon. So the table is not alphabetical by site, but that's fine as long as all of the "sampled_reef" vectors reflect this, so that reef_sizes, distance, and sampled_reefs match up by row/col index 
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- unique(fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")[
    , .(site, year=end_year, prop_anem_samp=total_prop_hab_sampled_anems_tidied)][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , prop_anem_samp := sum(prop_anem_samp), by=c("site", "year")], by=c("site", "year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & year %in% c(2012, 2013, 2014), prop_anem_samp :=1][site=="Caridad Proper" & year %in% c(2013, 2014), prop_anem_samp :=1]

### List of source locations
SitesSource <- Centroids

### List of destination locations
SitesDest <- Centroids

DistMatm <- distm(SitesSource[,c('lon','lat')], SitesSource[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- DistMatm*10^-3
#read in the reef areas for the kernel fitting
Area <- fread("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels_test.csv") %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6)# %>%
    #select(kmsq) #need to uncomment for functions to work
Area$site <- gsub("_ten_per_cover", "", Area$site, fixed=TRUE)

reef_sizes <- as.matrix(Area$kmsq)

#make a site index table, use this for Sampled_reefs input in kernel fitting
SiteIndex <- unique(Centroids %>% arrange(site), by="site")[, index := .I] #add the row number as the unique site index, leave CAI in if fitting a kernel 
SiteIndexBioPhys <- unique(Centroids %>% arrange(site), by="site")[site != "CAI"][, index := .I] #add the row number as the unique site index, take CAI out for biophysical likelihood function

#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0
#setnames(SurveyData, c("PropAnemSamp", "TotalAnems"), c("prop_anem_samp", "total_anems"))
#setkey(SurveyData, site)
#check all sites are represented in centroids and area (and indirectly distances, which comes from centroids)
#Area[site %!in% centroids$site] #should be nothing

#Allison's abundance time series data 
#download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/master/Data/Script_outputs/females_df_F.RData?raw=true", destfile = "~/oceanography/empirical_data/genetics/females_df_F.RData")
load("~/oceanography/empirical_data/genetics/females_df_F.RData")
Abundance <- as.data.table(females_df_F)
setnames(Abundance, "nF", "num_females")
Abundance <- unique(Abundance[site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
            , num_females := sum(num_females), by=c("site", "year")], by=c("site", "year"))
#join the survey sampling tables together
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year, site)][
    is.na(n_offs_gen), n_offs_gen := 0]#change NA's to 0


SurveyData <- Abundance[, c("year", "site", "num_females")][SurveyData, on=.(year, site)]#join in Allison's estimate of female abundance. There are NA values, but that's okay we can figure those out when we start thinking about incorporating uncertainty in this
#quick check that all components are in the same, alphabetical order
sum(which(SiteIndex$site==Area$site)==FALSE) #needs to be 0!! sites have to be in the same order
sum(which(Area$site==Centroids$site)==FALSE) #needs to be 0!! sites have to be in the same order

#read in genetic parentage matrices for each time frame MATCHING DIMENSIONS WITH BIOPHYSICAL FOR BIOPHYSICAL LIKELIHOOD- NOT KERNEL FITTING
GenMat2012_4 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012-14ForROMSComp.csv"))
GenMat2012 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2012ForROMSComp.csv"))
GenMat2013 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2013ForROMSComp.csv"))
GenMat2014 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrix2014ForROMSComp.csv"))
GenMatNEM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixNEM2012-14ForROMSComp.csv"))
GenMatSWM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210625_ParentageMatrixSWM2012-14ForROMSComp.csv"))

#read in genetic parentage matrices for each time frame FOR KERNEL FITTING
KernelGenMat2012_4 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv"))
KernelGenMat2012 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv"))
KernelGenMat2013 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv"))
KernelGenMat2014 <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv"))
KernelGenMatNEM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv"))
KernelGenMatSWM <- as.matrix(fread(file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv"))

#read in biophysical data for biophysical likelihood
FullBiophysMatNorm <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012-14ForROMSComp.csv"))
AnnualBiophysMatNorm2012 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2012ForROMSComp.csv"))
AnnualBiophysMatNorm2013 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2013ForROMSComp.csv"))
AnnualBiophysMatNorm2014 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrix2014ForROMSComp.csv"))
MonsoonBiophysMatNormNEM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixNEM2012-14ForROMSComp.csv"))
MonsoonBiophysMatNormSWM <-as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysNormConnMatrixSWM2012-14ForROMSComp.csv"))

#read in biophysical data for KERNEL FITTING
FullBiophysMat <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2012-14ForROMSComp.csv"))
AnnualBiophysMat2012 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2012ForROMSComp.csv"))
AnnualBiophysMat2013 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2013ForROMSComp.csv"))
AnnualBiophysMat2014 <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrix2014ForROMSComp.csv"))
MonsoonBiophysMatNEM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixNEM2012-14ForROMSComp.csv"))
MonsoonBiophysMatSWM <- as.matrix(fread(file="~/oceanography/script_output/ROMSDataTables/20210701_BioPhysParentageMatrixSWM2012-14ForROMSComp.csv"))

[1] 0

[1] 0

In [23]:
biophys_par_data2012

0.0000000,36.59466,3.0328732,3.7812405,39.544910,35.144215,36.419898,40.865998,7.7170642,15.862935,...,2.6540921,12.7521560,11.9430714,11.3862533,24.7874862,10.8525552,6.9584971,23.276705,13.915290,2.1280885
36.5946593,0.00000,37.5112742,37.6253510,32.701393,28.926277,27.351389,33.695741,38.1057602,39.049765,...,36.2377630,38.6352916,38.8423125,38.7832324,37.2650016,38.8627857,38.1614224,37.940511,38.551899,36.4035501
3.0328732,37.51127,0.0000000,0.7611792,37.871146,33.627822,35.066563,39.166201,4.7583636,13.024371,...,5.6803278,9.8703541,9.0230795,8.4572682,22.3207572,7.9035693,3.9724282,20.716564,11.069808,5.1471170
3.7812405,37.62535,0.7611792,0.0000000,37.352525,33.148293,34.623834,38.640953,3.9999295,12.278269,...,6.4329063,9.1192352,8.2669730,7.7002819,21.6392752,7.1445494,3.2116192,20.017227,10.323743,5.9014240
39.5449097,32.70139,37.8711462,37.3525247,0.000000,4.863776,5.420621,1.370390,34.4925112,28.304747,...,41.3312751,30.6573228,31.5336407,31.9607107,18.7098394,32.4885497,35.1757138,20.679329,29.579123,41.0485538
35.1442145,28.92628,33.6278217,33.1482933,4.863776,0.000000,2.533995,6.223798,30.5139243,24.963682,...,36.8276566,27.0362084,27.8664339,28.2505730,16.1946825,28.7460208,31.1571402,18.037808,26.040679,36.5673271
36.4198977,27.35139,35.0665635,34.6238340,5.420621,2.533995,0.000000,6.580623,32.1779180,27.009579,...,37.9754549,28.9363496,29.7373991,30.0955838,18.5624726,30.5699476,32.7894955,20.362496,27.988733,37.7441889
40.8659979,33.69574,39.1662013,38.6409535,1.370390,6.223798,6.580623,0.000000,35.7417602,29.438628,...,42.6682007,31.8425287,32.7266049,33.1614679,19.7145272,33.6949350,36.4318492,21.697643,30.749828,42.3820949
7.7170642,38.10576,4.7583636,3.9999295,34.492511,30.513924,32.177918,35.741760,0.0000000,8.315917,...,10.3654013,5.1402209,4.2700861,3.7016111,17.9802829,3.1454389,0.8152421,16.270087,6.366148,9.8446575
15.8629348,39.04977,13.0243709,12.2782690,28.304747,24.963682,27.009579,29.438628,8.3159171,0.000000,...,18.4582391,3.1819446,4.1010325,4.6708512,10.2978939,5.2677875,9.1296641,8.395073,1.954587,17.9621025
24.4585460,37.82302,21.9388356,21.2468135,19.483034,16.994224,19.361938,20.477052,17.5336836,9.713904,...,26.8648033,12.6833762,13.6344936,14.1717075,0.7997638,14.7679554,18.3347502,1.322755,11.440425,26.4270338


In [25]:
biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , .(site)], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))


ERROR: Error in `[<-`(`*tmp*`, i, j, value = structure(c(NA_real_, NA_real_, : subscript out of bounds


In [32]:


#fit the kernels, get the biophysical data together
biophys_par_data2012 <- list(Distances=Distances, Assignments=AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(FullBiophysMat), ncol=1, 1))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))



In [33]:

#make a summary table for all time frames of simulation data
SimKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 7))
setDT(SimKernels)
setnames(SimKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90"))
#SimKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[1] <- "annual"
SimKernels$time_id[1] <- "2012"
SimKernels$k[1] <-  BestK2012
SimKernels$theta[1] <- BestTheta2012
SimKernels$mdd[1] <- MDD2012
SimKernels$med[1] <- Med2012
SimKernels$dist90[1] <- Dist90_2012

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[2] <- "annual"
SimKernels$time_id[2] <- "2013"
SimKernels$k[2] <-  BestK2013
SimKernels$theta[2] <- BestTheta2013
SimKernels$mdd[2] <- MDD2013
SimKernels$med[2] <- Med2013
SimKernels$dist90[2] <- Dist90_2013

BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[3] <- "annual"
SimKernels$time_id[3] <- "2014"
SimKernels$k[3] <-  BestK2014
SimKernels$theta[3] <- BestTheta2014
SimKernels$mdd[3] <- MDD2014
SimKernels$med[3] <- Med2014
SimKernels$dist90[3] <- Dist90_2014

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[4] <- "interannual"
SimKernels$time_id[4] <- "2012-4"
SimKernels$k[4] <-  BestK2012_4
SimKernels$theta[4] <- BestTheta2012_4
SimKernels$mdd[4] <- MDD2012_4
SimKernels$med[4] <- Med2012_4
SimKernels$dist90[4] <- Dist90_2012_4

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[5] <- "season"
SimKernels$time_id[5] <- "NEM"
SimKernels$k[5] <-  BestKNEM
SimKernels$theta[5] <- BestThetaNEM
SimKernels$mdd[5] <- MDDNEM
SimKernels$med[5] <- MedNEM
SimKernels$dist90[5] <- Dist90_NEM

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

SimKernels$time_scale[6] <- "season"
SimKernels$time_id[6] <- "SWM"
SimKernels$k[6] <-  BestKSWM
SimKernels$theta[6] <- BestThetaSWM
SimKernels$mdd[6] <- MDDSWM
SimKernels$med[6] <- MedSWM
SimKernels$dist90[6] <- Dist90_SWM

#fwrite(SimKernels, file="~/oceanography/script_output/KernelFits/SimulationKernelFits.csv")


SimKernels


time_scale,time_id,k,theta,mdd,med,dist90
annual,2012,-2.1643785,4.9907350,4.208282,4.01,7.88
annual,2013,-0.4612801,0.6297245,4.207437,2.32,10.38
annual,2014,0.2830958,0.5275078,3.645378,1.78,9.16
interannual,2012-4,-0.1038335,0.5761347,3.901122,2.03,9.72
season,NEM,-0.2254188,0.6073513,3.710193,2.00,9.19
season,SWM,0.6027193,0.4164045,7.886669,3.17,20.02


In [58]:
download.file(url = "https://github.com/pinskylab/Clownfish_persistence/blob/b5b4c1ab8e089c3de85e7108118f3ae493ceb5d2/Data/Script_outputs/cumulative_prop_hab_sampled_by_site.RData?raw=true", destfile = "~/parentage/r_data/old_prop_samp.Rdata")  

load("~/parentage/r_data/old_prop_samp.Rdata")
OldPropSamp <- cumulative_prop_hab_sampled_by_site


In [87]:
cumulative_prop_hab_sampled_by_site %>% filter(end_year==2012)

site,method,time_frame,end_year,total_possible_sample_anems,total_anems_sampled,total_anems_sampled_tidied,total_prop_hab_sampled_anems,total_prop_hab_sampled_anems_tidied
Cabatoan,metal tags,2012,2012,26,11,11,0.42307692,0.4230769
Caridad Cemetery,metal tags,2012,2012,4,0,0,0.00000000,0.0000000
Caridad Proper,metal tags,2012,2012,0,0,0,NaN,0.0000000
Elementary School,metal tags,2012,2012,8,0,0,0.00000000,0.0000000
Gabas,metal tags,2012,2012,9,0,0,0.00000000,0.0000000
Haina,metal tags,2012,2012,104,0,0,0.00000000,0.0000000
Hicgop South,metal tags,2012,2012,18,0,0,0.00000000,0.0000000
Magbangon,metal tags,2012,2012,105,5,5,0.04761905,0.9187675
Magbangon,metal tags,2012,2012,34,14,14,0.41176471,0.9187675
Palanas,metal tags,2012,2012,137,40,40,0.29197080,0.2919708


In [80]:
OldPropSamp[is.nan(OldPropSamp$total_prop_hab_sampled_anems_tidied), total_prop_hab_sampled_anems_tidied := 0][
    is.infinite(OldPropSamp$total_prop_hab_sampled_anems_tidied), total_prop_hab_sampled_anems_tidied := 1][
    site=="Sitio Lonas" & end_year== 2012, total_prop_hab_sampled_anems_tidied :=1][
    site=="Caridad Proper" & end_year== 2013, total_prop_hab_sampled_anems_tidied :=1][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , total_prop_hab_sampled_anems_tidied := sum(total_prop_hab_sampled_anems_tidied), by=c("site", "end_year")]

In [81]:
setDT(OldPropSamp)

OldPropSamp<- unique(OldPropSamp[is.nan(OldPropSamp$total_prop_hab_sampled_anems_tidied), total_prop_hab_sampled_anems_tidied := 0][
    is.infinite(OldPropSamp$total_prop_hab_sampled_anems_tidied), total_prop_hab_sampled_anems_tidied := 1][
    site=="Sitio Lonas" & end_year== 2012, total_prop_hab_sampled_anems_tidied :=1][
    site=="Caridad Proper" & end_year== 2013, total_prop_hab_sampled_anems_tidied :=1][ #select and rename columns with the tideied data to use
    site %like% "Magbangon", site := "Magbangon"][ #collapse Magbangon values
    , total_prop_hab_sampled_anems_tidied := sum(total_prop_hab_sampled_anems_tidied), by=c("site", "end_year")], by=c("site", "end_year"))[ #collapse magbangons to match ROMS data
    site=="Sitio Lonas" & end_year %in% c(2012, 2013, 2014), total_prop_hab_sampled_anems_tidied :=1][site=="Caridad Proper" & end_year %in% c(2013, 2014), total_prop_hab_sampled_anems_tidied :=1]


In [82]:
setnames(OldPropSamp, c("end_year", "total_prop_hab_sampled_anems_tidied"), c("year", "prop_anem_samp"))


In [36]:

prop_samp12 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp12.csv", header=FALSE))
prop_samp13 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp13.csv", header=FALSE))
prop_samp14 <- as.matrix(read.csv("~/parentage/kernel_fitting/1340_loci/input/prop_samp14.csv", header=FALSE))


In [86]:
OldPropSamp[year==2012 & prop_anem_samp >0, .(site, prop_anem_samp)]
prop_samp12

site,prop_anem_samp
Cabatoan,0.4230769
Magbangon,0.9187675
Palanas,0.2919708
Poroc Rose,1.0000000
Poroc San Flower,1.0000000
San Agustin,0.9411765
Sitio Lonas,1.0000000
Visca,1.0000000
Wangag,0.1790541


V1
0.42307692
0.04761905
0.41176471
0.29197080
1.00000000
1.00000000
0.94117647
1.00000000
1.00000000
0.17905405


In [ ]:
0.41176471+

In [91]:


#testing
par_data2012 <- list(Distances=Distances, Assignments=KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(OldPropSamp[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))



In [ ]:

#fit the kernels, get the genetic data together
par_data2012 <- list(Distances=Distances, Assignments=KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMat2012), ncol=1, 1))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMat2013), ncol=1, 1))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMat2014), ncol=1, 1))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMat2012_4), ncol=1, 1))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMatNEM), ncol=1, 1))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(KernelGenMatSWM), ncol=1, 1))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))


In [93]:
#make a summary table for all time frames of genetic data
GenKernels <- as.data.frame(matrix(NA, nrow = 6, ncol = 7))
setDT(GenKernels)
setnames(GenKernels, c("time_scale", "time_id", "k", "theta", "mdd", "med", "dist90"))
#GenKernels <- data.table(time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[1] <- "annual"
GenKernels$time_id[1] <- "2012"
GenKernels$k[1] <-  BestK2012
GenKernels$theta[1] <- BestTheta2012
GenKernels$mdd[1] <- MDD2012
GenKernels$med[1] <- Med2012
GenKernels$dist90[1] <- Dist90_2012

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[2] <- "annual"
GenKernels$time_id[2] <- "2013"
GenKernels$k[2] <-  BestK2013
GenKernels$theta[2] <- BestTheta2013
GenKernels$mdd[2] <- MDD2013
GenKernels$med[2] <- Med2013
GenKernels$dist90[2] <- Dist90_2013

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[3] <- "annual"
GenKernels$time_id[3] <- "2014"
GenKernels$k[3] <-  BestK2014
GenKernels$theta[3] <- BestTheta2014
GenKernels$mdd[3] <- MDD2014
GenKernels$med[3] <- Med2014
GenKernels$dist90[3] <- Dist90_2014

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[4] <- "interannual"
GenKernels$time_id[4] <- "2012-4"
GenKernels$k[4] <-  BestK2012_4
GenKernels$theta[4] <- BestTheta2012_4
GenKernels$mdd[4] <- MDD2012_4
GenKernels$med[4] <- Med2012_4
GenKernels$dist90[4] <- Dist90_2012_4

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[5] <- "season"
GenKernels$time_id[5] <- "NEM"
GenKernels$k[5] <-  BestKNEM
GenKernels$theta[5] <- BestThetaNEM
GenKernels$mdd[5] <- MDDNEM
GenKernels$med[5] <- MedNEM
GenKernels$dist90[5] <- Dist90_NEM

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)

GenKernels$time_scale[6] <- "season"
GenKernels$time_id[6] <- "SWM"
GenKernels$k[6] <-  BestKSWM
GenKernels$theta[6] <- BestThetaSWM
GenKernels$mdd[6] <- MDDSWM
GenKernels$med[6] <- MedSWM
GenKernels$dist90[6] <- Dist90_SWM

#fwrite(GenKernels, file="~/oceanography/script_output/KernelFits/GeneticKernelFits.csv")


GenKernels

kernel2012_14

time_scale,time_id,k,theta,mdd,med,dist90
annual,2012,-3.3135464,3.8081144,13.48898,12.55,25.79
annual,2013,-3.3903070,8.0000000,14.28073,13.98,26.10
annual,2014,1.9702749,0.2583811,81.87307,18.63,196.08
interannual,2012-4,-0.2240313,0.4358496,14.17058,5.94,35.96
season,NEM,0.7797079,0.3697049,13.39359,4.78,33.92
season,SWM,-3.4641490,8.0000000,15.37516,15.05,28.10


year,k,theta,mdd,med,dist90
2012-14,-0.7501654,0.7857855,3.222821,2.01,7.71


In [92]:
GenKernels

time_scale,time_id,k,theta,mdd,med,dist90
annual,2012,-3.3982819,4.6497158,14.49377,13.74,27.27
annual,2013,-3.3612735,5.8979020,13.87613,13.38,25.71
annual,2014,1.8369101,0.2438280,177.45992,36.94,417.46
interannual,2012-4,-0.4359911,0.4295055,18.89236,7.81,47.95
season,NEM,0.6632164,0.3542678,19.98737,6.82,50.49
season,SWM,-3.4901989,8.0000000,15.78094,15.45,28.84


In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_data2012$Distances, Assignments=biophys_par_data2012$Assignments, Sampled_reefs=biophys_par_data2012$Sampled_reefs, Reef_sizes=biophys_par_data2012$Reef_sizes, Adult_sample_proportions=biophys_par_data2012$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2012.csv", row.names=T, quote=FALSE)

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_data2013$Distances, Assignments=biophys_par_data2013$Assignments, Sampled_reefs=biophys_par_data2013$Sampled_reefs, Reef_sizes=biophys_par_data2013$Reef_sizes, Adult_sample_proportions=biophys_par_data2013$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2013.csv", row.names=T, quote=FALSE)

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_data2014$Distances, Assignments=biophys_par_data2014$Assignments, Sampled_reefs=biophys_par_data2014$Sampled_reefs, Reef_sizes=biophys_par_data2014$Reef_sizes, Adult_sample_proportions=biophys_par_data2014$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2014.csv", row.names=T, quote=FALSE)

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_data2012_4$Distances, Assignments=biophys_par_data2012_4$Assignments, Sampled_reefs=biophys_par_data2012_4$Sampled_reefs, Reef_sizes=biophys_par_data2012_4$Reef_sizes, Adult_sample_proportions=biophys_par_data2012_4$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysical2012_4.csv", row.names=T, quote=FALSE)

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_dataNEM$Distances, Assignments=biophys_par_dataNEM$Assignments, Sampled_reefs=biophys_par_dataNEM$Sampled_reefs, Reef_sizes=biophys_par_dataNEM$Reef_sizes, Adult_sample_proportions=biophys_par_dataNEM$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysicalNEM.csv", row.names=T, quote=FALSE)

In [ ]:
# use a grid search to find k that minimizes the log likelihood
k_eval <- c(seq(from=-10, to=10, by=0.01))
theta_eval <- c(seq(from=0.1, to=5, by=.01))
nll_matrix <- matrix(data=NA, nrow=length(k_eval), ncol=length(theta_eval), 
                     dimnames=list(k_eval, theta_eval))
pb <- txtProgressBar(min = 0, max = length(k_eval), style = 3)#7 years in each interation

#Begin grid search: i <- 1; j <- 1
for(i in 1:length(k_eval)){
  k <- k_eval[i]
  for(j in 1:length(theta_eval)){
    theta <- theta_eval[j]
    nll_matrix[i,j] <- LL_kt_grid(k=k, theta=theta, Distances=biophys_par_dataSWM$Distances, Assignments=biophys_par_dataSWM$Assignments, Sampled_reefs=biophys_par_dataSWM$Sampled_reefs, Reef_sizes=biophys_par_dataSWM$Reef_sizes, Adult_sample_proportions=biophys_par_dataSWM$Adult_sample_proportions)
  }
    setTxtProgressBar(pb, i)

}


close(pb)

max(nll_matrix, na.rm = T)
min(nll_matrix, na.rm = T)

sum(is.na(nll_matrix)==T)
best_params_index <- which(nll_matrix == min(nll_matrix, na.rm = T), arr.ind=TRUE)
best_params <- c(k_eval[best_params_index[1]], theta_eval[best_params_index[2]])
best_params


#write profile results
write.csv(nll_matrix, file="~/oceanography/script_output/KernelFits/LikelihoodProfileBiophysicalSWM.csv", row.names=T, quote=FALSE)

In [ ]:
##lay out all the pieces
#sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)])
#pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
#BioPhysMat <- as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]) #source normalized biophysical connectivity matrix. In Eqn. S3.4, this is m ajt/r a (*should it be r at? As in all particles released in time period t?)
#prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function
#unassigned_vec <- as.matrix(GenMat2012_4[nrow(GenMat2012_4),])#from genetic parentage data- a vector of the number of unassigned recruits at each destination reef in the system- we only have this for all sampled reefs.... what should the dimensions be?*
#Assignments <- GenMat2012_4[1:nrow(GenMat2012_4)-1,]
#
Data2012_4 <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012_4[nrow(GenMat2012_4),]))

Data2012 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
 Data2014 <- list(BioPhysMat=as.matrix(AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

 DataNEM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM <- list(BioPhysMat=as.matrix(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012_4[1:nrow(GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))

#calculate likelihood of our observed data given the biophysical simulation data- we want the highest likelihood, which is the smallest negative log likelihood
data.table(time_frame= c("interannual", "annual", "monsoonal"), neg_LL=c(neg_LL_biophys(Data2012_4), neg_LL_biophys(Data2012) + neg_LL_biophys(Data2013) + neg_LL_biophys(Data2014), neg_LL_biophys(DataNEM) + neg_LL_biophys(DataSWM)))[order(-neg_LL)]


Data2012_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2012[1:nrow(GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2012[nrow(GenMat2012),]))
Data2013_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2013[1:nrow(GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2013[nrow(GenMat2013),]))
Data2014_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMat2014[1:nrow(GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMat2014[nrow(GenMat2014),]))

neg_LL_biophys(Data2012_test)+neg_LL_biophys(Data2013_test)+neg_LL_biophys(Data2014_test)

 DataNEM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatNEM[1:nrow(GenMatNEM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatNEM[nrow(GenMatNEM),]))

 DataSWM_test <- list(BioPhysMat=as.matrix(FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=GenMatSWM[1:nrow(GenMatSWM)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(GenMatSWM[nrow(GenMatSWM),]))
neg_LL_biophys(DataNEM_test)+neg_LL_biophys( DataSWM_test)

sum(colSums(MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]))
sum(colSums(MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]))

